---
# Safety and Security - Ingest Video Streams

To avoid the need for a set of live cameras for this demo, we play back video from a series of JPEG files on disk
and write each video frame to SDP.

---

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import OutputStream, ImageFileSequenceLoader
import os

### Define Pravega stream parameters

In [ ]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
stream = 'safety-and-security-video'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [ ]:
#pravega_client.CreateScope(pravega.pb.CreateScopeRequest(scope=scope))

### Create Pravega stream

In [ ]:
output_stream = OutputStream(pravega_client, scope, stream)
#output_stream.delete_stream()
output_stream.create_stream()
output_stream.truncate_stream()

In [ ]:
# Truncate object detector output stream
detected_stream = OutputStream(pravega_client, scope, 'multi-video-grid-output')
#detected_stream.delete_stream()
detected_stream.create_stream(min_num_segments=1)
detected_stream.truncate_stream()

### Ingest JPEG images from files

In [ ]:
data_dir = '../../data/'
camera_filespecs = [
    data_dir + 'tollway-system-in-a-multi-lane-expressway-3324160/*.jpg',
    data_dir + 'traffic-intersection-4188267/*.jpg',
    data_dir + 'aerial-footage-of-vehicular-traffic-of-a-busy-street-intersection-at-night-3048225/*.jpg',
    data_dir + 'people-walking-on-the-street-of-a-shopping-center-3283572/*.jpg',
#     data_dir + 'a-cargo-truck-maneuvering-turns-in-a-manufacturing-plant-2711276/*.jpg',
#     data_dir + 'aerial-view-of-a-freeway-1472012/*.jpg',
#     data_dir + 'drone-footage-of-a-city-s-business-district-2835994/*.jpg',
#     data_dir + 'drone-footage-of-a-manufacturing-plant-3061260/*.jpg',
#     data_dir + 'rail-freight-trains-in-transit-4066355/*.jpg',
#     data_dir + 'aerial-view-of-bridge-and-river-2292093/*.jpg',
#     data_dir + 'virat/aerial/09152008flight2tape1_10/*.jpg',
#     data_dir + 'virat/ground/VIRAT_S_000200_01_000226_000268/*.jpg',
#     data_dir + 'virat/ground/VIRAT_S_000201_08_001652_001838/*.jpg',
#     data_dir + 'virat/ground/VIRAT_S_000205_03_000860_000922/*.jpg',
#     data_dir + 'virat/aerial/09172008flight1tape1_5/*.jpg',
#     data_dir + 'virat/aerial/09152008flight2tape3_1/*.jpg',
#     data_dir + 'virat/aerial/09152008flight2tape1_7/*.jpg',
]
input_fps = 15
step = 1
fps = input_fps / step

In [ ]:
loader = ImageFileSequenceLoader(scope, stream, camera_filespecs, fps, step=step)

In [ ]:
events_to_write = loader.event_generator()
output_stream.write_events(events_to_write)